## Visão Geral

Nesta clusterização os dados são agregados por município para a formação de possíveis clusters regionais. Todos os dados agregados são proporcionais aos casos do município para evitar que municípios com mais casos estraguem a análise por desbalanceamento.<br>

Os seguintes dados serão agregados:
 - Mediana de idade (ou média)
 - Porcentagem de hospitalização
 - Porcentagem de óbito por agravo
 - Porcentagem de PPI
 - Porcentagem de brancos (amarelos somam os 100% implicitamente pra evitar redundancia)
 - Porcentagem de homens (ou mulheres)
 - Porcentagem de gestantes
 - Porcentagem com comorbidades
 - Porcentagem com sintomas severos (nausea/vomito/dor-abdominal)
 - Porcentagem de dengue com sinais de alarme
 - Porcentagem de dengue grave


*Serão considerados somente casos confirmados: CLASSI_FIN = 10 || 11 || 12*<br>

## Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

## Limpeza e pré processamento

### Carregamento

In [ ]:
desired_cols = [
  "SG_UF",      # Localidade
  "ID_MN_RESI", 

  "ANO_NASC",   # Sociais
  "CS_SEXO",
  "CS_RACA",
  "CS_GESTANT",

  "VOMITO",     # Sintomas
  "NAUSEA",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",

  "HOSPITALIZ", # Desfecho
  "CLASSI_FIN", 
  "EVOLUCAO",
]

# Carregando dataset
DATA_PATH = "./data/dbc/DENGBR24.csv"

chunks = []

for chunk in pd.read_csv(DATA_PATH, low_memory=False, usecols=desired_cols, chunksize=500_000):
  filtered = chunk[
    (chunk["CLASSI_FIN"].isin([10, 11, 12]))  &
    (chunk["ANO_NASC"].notna())               &   
    (chunk["ANO_NASC"] > 1924)                &   
    (chunk["CS_SEXO"].isin(["M","F"]))        &
    (chunk["CS_RACA"].isin([1, 2, 3, 4, 5]))              
  ]
  chunks.append(filtered)

df = pd.concat(chunks, ignore_index=True)
df.head()

In [ ]:
df.info()

### Tratamento dos dados

In [ ]:
df_treated = df.copy()

In [ ]:

df_treated["IDADE"] = 2024 - df_treated["ANO_NASC"]

df_treated["SINTOMAS_SEVEROS"] = (
  (df_treated["NAUSEA"] == 1) | 
  (df_treated["VOMITO"] == 1) | 
  (df_treated["DOR_RETRO"] == 1)
).astype(int)

df_treated["COMORBIDADES"] = (
  (df_treated["DIABETES"] == 1)   | 
  (df_treated["HEMATOLOG"] == 1)  |
  (df_treated["HEPATOPAT"] == 1)  |
  (df_treated["RENAL"] == 1)      |
  (df_treated["HIPERTENSA"] == 1) |
  (df_treated["ACIDO_PEPT"] == 1) |
  (df_treated["AUTO_IMUNE"] == 1)
).astype(int)

df_treated["PPI"] = df_treated["CS_RACA"].isin([2, 4, 5]).astype(int)
df_treated["BRANCO"] = (df_treated["CS_RACA"] == 1).astype(int)
df_treated["MASCULINO"] = (df_treated["CS_SEXO"] == 'M').astype(int)
df_treated["OBITO_AGRAVO"] = (df_treated["EVOLUCAO"] == 2).astype(int)
df_treated["GESTANTE"] = df_treated["CS_GESTANT"].between(1, 4).astype(int)
df_treated["DENGUE_SA"] = (df_treated["CLASSI_FIN"] == 11).astype(int)
df_treated["DENGUE_GRAVE"] = (df_treated["CLASSI_FIN"] == 12).astype(int)
df_treated["HOSPITALIZACAO"] = (df_treated["HOSPITALIZ"] == 1).astype(int)

In [ ]:
clustering_cols = [
  "ID_MN_RESI", "IDADE", "SINTOMAS_SEVEROS", "COMORBIDADES", "PPI", "BRANCO", "MASCULINO",
  "OBITO_AGRAVO", "GESTANTE", "DENGUE_SA", "DENGUE_GRAVE", "HOSPITALIZACAO"
]
df_clustering = df_treated[clustering_cols]
df_clustering.head()

In [ ]:
df_agg = df_clustering.groupby("ID_MN_RESI").agg({
  "IDADE": "median",
  "HOSPITALIZACAO": "mean",
  "OBITO_AGRAVO": "mean",
  "PPI": "mean",
  "BRANCO": "mean",
  "MASCULINO": "mean",
  "GESTANTE": "mean",
  "COMORBIDADES": "mean",
  "SINTOMAS_SEVEROS": "mean",
  "DENGUE_SA": "mean",
  "DENGUE_GRAVE": "mean"
}).rename(columns={
  "IDADE": "mediana_idade",
  "HOSPITALIZACAO": "%_hospitalizacao",
  "OBITO_AGRAVO": "%_obito_agravo",
  "PPI": "%_ppi",
  "BRANCO": "%_brancos",
  "MASCULINO": "%_masculino",
  "GESTANTE": "%_gestante",
  "COMORBIDADES": "%_comorbidades",
  "SINTOMAS_SEVEROS": "%_sintomas_severos",
  "DENGUE_SA": "%_dengue_sinais_alarme",
  "DENGUE_GRAVE": "%_dengue_grave"
}).reset_index()

df_agg = df_agg.rename(columns={"ID_MN_RESI": "municipio"})
df_agg.head()

## Pré clusterização

### Correlação

In [ ]:
df_corr = df_agg.drop(columns=["municipio"])
corr_matrix = df_corr.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlação entre Variáveis Agregadas")
plt.tight_layout()
plt.show()

### Normalização

In [ ]:
municipios = df_agg["municipio"]

df_final = df_agg.drop(columns=["%_brancos", "municipio"])
df_final.head()

In [ ]:
scaler = StandardScaler()
features_norm = scaler.fit_transform(df_final)

### Escolha do K do KMeans

#### Inércia

In [ ]:
inertia = []
K_range = range(1, 18)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=1)
    kmeans.fit(features_norm)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8,5))
plt.plot(K_range, inertia, 'bo-')
plt.xlabel('Número de clusters K')
plt.ylabel('Inércia (Soma das distâncias quadradas)')
plt.title('Método do Cotovelo para escolher K')
plt.show()

#### Coeficiente de silhueta v1

In [ ]:
silhouette_scores = []
K_range = range(2, 12)  # Silhouette não é definido para K=1

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=1)
    labels = kmeans.fit_predict(features_norm)
    score = silhouette_score(features_norm, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(8,5))
plt.plot(K_range, silhouette_scores, 'bo-')
plt.xlabel('Número de clusters K')
plt.ylabel('Índice Silhouette')
plt.title('Avaliação do K usando Índice Silhouette')
plt.show()

#### Coeficiente de silhueta v2

In [ ]:
# Trecho adaptado de 
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1) = plt.subplots(1)
    fig.set_size_inches(8, 5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(features_norm) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=1)
    cluster_labels = clusterer.fit_predict(features_norm)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(features_norm, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(features_norm, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

## Clusterização

### KMeans (melhor resultado)

In [ ]:
kmeans = KMeans(n_clusters=9, random_state=1)
clusters = kmeans.fit_predict(features_norm)

In [ ]:
# Para visualização em 2D
pca_2d = PCA(n_components=2)
features_pca_2d = pca_2d.fit_transform(features_norm)

# Para visualização em 3D
pca_3d = PCA(n_components=3)
features_pca_3d = pca_3d.fit_transform(features_norm)

In [ ]:
plt.figure(figsize=(8,6))
scatter = plt.scatter(features_pca_2d[:, 0], features_pca_2d[:, 1], c=clusters, cmap='rocket')
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title("Clusters com KMeans (2D PCA)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.colorbar(scatter, label='Cluster')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df_clustered = df_agg.copy()

df_clustered["cluster"] = clusters
df_clustered.head()

In [ ]:
df_stats = df_clustered.drop(columns=['municipio', 'mediana_idade'])
stats_por_cluster = df_stats.groupby('cluster').mean()

stats_por_cluster = stats_por_cluster * 100
plt.figure(figsize=(12, 6))
sns.heatmap(stats_por_cluster.T, annot=stats_por_cluster.T.round(2).astype(str) + '%', fmt="", cmap='rocket_r')
plt.title("Média das Variáveis por Cluster")
plt.xlabel("Clusters")
plt.ylabel("Variáveis")
plt.tight_layout()
plt.show()

In [ ]:
media_mediana_idade = df_clustered.groupby('cluster')['mediana_idade'].mean()
media_mediana_idade

plt.figure(figsize=(10,6))
sns.barplot(x=media_mediana_idade.index, y=media_mediana_idade.values, palette="viridis")
plt.title("Média da Mediana de Idades por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Média da Mediana de Idade")
plt.xticks(media_mediana_idade.index)
plt.ylim(0, media_mediana_idade.max() + 5)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Opcional: mostrar os valores em cima das barras
for i, v in enumerate(media_mediana_idade.values):
    plt.text(i, v + 0.5, f"{v:.2f}", ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# Converter para string e extrair os 2 primeiros dígitos como código do estado
df_clustered['codigo_uf'] = df_clustered['municipio'].astype(int).astype(str).str[:2]
df_clustered['codigo_uf'] = df_clustered['codigo_uf'].astype(int)

codigo_uf_para_nome = {
    11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO',
    21: 'MA', 22: 'PI', 23: 'CE', 24: 'RN', 25: 'PB', 26: 'PE', 27: 'AL', 
    28: 'SE', 29: 'BA', 31: 'MG', 32: 'ES', 33: 'RJ', 35: 'SP', 41: 'PR', 
    42: 'SC', 43: 'RS', 50: 'MS', 51: 'MT', 52: 'GO', 53: 'DF'
}

df_clustered['estado'] = df_clustered['codigo_uf'].map(codigo_uf_para_nome)
contagem_estado = df_clustered.groupby(['cluster', 'estado']).size().unstack(fill_value=0)
percentual_estado = contagem_estado.div(contagem_estado.sum(axis=1), axis=0) * 100
plt.figure(figsize=(15, 8))
sns.heatmap(percentual_estado, annot=True, fmt='.2f', cmap='rocket_r')
plt.title('Prevalência percentual por estado em cada cluster')
plt.xlabel('Estado')
plt.ylabel('Cluster')
plt.tight_layout()
plt.show()


In [ ]:
# GeoJSON com os estados
brasil = gpd.read_file("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson")

# Renomear para garantir que a coluna de estado tenha o mesmo nome do seu DataFrame
brasil = brasil.rename(columns={"name": "estado"})
brasil["estado"] = brasil["estado"].str.upper()

In [ ]:
percentual_estado

In [ ]:
# Supondo que df_prevalencia seja seu DataFrame atual
df_long = percentual_estado.reset_index().melt(id_vars='cluster', var_name='estado', value_name='prevalencia')

In [ ]:
nome_to_sigla = {
    'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA',
    'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
    'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG',
    'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI',
    'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP',
    'Sergipe': 'SE', 'Tocantins': 'TO'
}

brasil['estado'] = brasil['sigla'].map(nome_to_sigla)
# brasil.columns

In [ ]:
cltrs = df_long['cluster'].unique()

for cluster_id in cltrs:
    dados_cluster = df_long[df_long['cluster'] == cluster_id]
    
    # Junta com o mapa
    mapa = brasil.merge(dados_cluster, left_on='sigla' ,right_on="estado", how="left")

    # Plot
    fig, ax = plt.subplots(figsize=(12, 10))
    mapa.plot(
        column="prevalencia",
        cmap="OrRd",
        linewidth=0.8,
        ax=ax,
        edgecolor='0.8',
        legend=True,
        missing_kwds={"color": "lightgrey", "label": "Sem dados"},
    )

    ax.set_title(f"Distribuição percentual de municípios por estado - Cluster {cluster_id}", fontsize=16)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

### Agrupamento Espectral (resultado insatisfatório)

#### Escolha do K

Calculando eigengap

In [ ]:
from sklearn.neighbors import kneighbors_graph
from scipy.sparse import csgraph
from numpy import linalg as LA

# 1 Construindo a matriz de adjacências do grafo de vizinhos mais próximos.
G = kneighbors_graph(features_norm, n_neighbors = 10, include_self = True)
A = 0.5 * (G + G.T)

# 2 Construindo a Laplaciana Normalizada
L = csgraph.laplacian(A, normed = True).todense()

# 3 Obtendo os autovalores da Laplaciana Normalizada
# Valores já estão ordenados em ordem crescente.
values, _ = LA.eigh(L)

# 4 Plotando os valores dos 'gaps' e escolhendo um k adequado.
plt.scatter([i for i in range(1, 21)], values[:20])
plt.xlabel('Índice do autovalor')
plt.ylabel('Autovalor');

#### Clusertização

In [ ]:
from sklearn.cluster import SpectralClustering

# Defina o número de clusters desejado
n_clusters = 2 # ou o número que você quiser testar

# Aplique o agrupamento espectral
spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=42)
labels_spectral = spectral.fit_predict(features_norm)

# Adicione os rótulos ao seu dataframe original
df_clustered['cluster_spectral'] = labels_spectral

In [ ]:
plt.scatter(features_pca_2d[:, 0], features_pca_2d[:, 1], c=labels_spectral, cmap='rocket')
plt.title("Clusters com Spectral Clustering")
plt.show()

In [ ]:
df_clustered.head()

In [ ]:
df_stats_spectral = df_clustered.drop(columns=['municipio', 'mediana_idade', "codigo_uf", "estado", "cluster"])
stats_por_cluster_spectral = df_stats_spectral.groupby('cluster_spectral').mean()

stats_por_cluster_spectral = stats_por_cluster_spectral * 100
plt.figure(figsize=(12, 6))
sns.heatmap(stats_por_cluster_spectral.T, annot=stats_por_cluster_spectral.T.round(2).astype(str) + '%', fmt="", cmap='rocket_r')
plt.title("Média das Variáveis por Cluster")
plt.xlabel("Clusters")
plt.ylabel("Variáveis")
plt.tight_layout()
plt.show()

### DBScan (resultado insatisfatório)

#### Escolha do eps

In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import numpy as np

k = 10  # igual ao min_samples
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(features_norm)
distances, indices = neighbors_fit.kneighbors(features_norm)

distances = np.sort(distances[:, -1])  # distância até o 10º vizinho

plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.title("Gráfico k-distance para escolha do eps")
plt.xlabel("Pontos ordenados")
plt.ylabel(f"Distância ao {k}º vizinho")
plt.grid(True)
plt.show()


#### Clusterização

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=4, min_samples=20)  # k = min_samples
labels = dbscan.fit_predict(features_norm)

df_clustered['cluster_dbscan'] = labels

In [ ]:
sns.countplot(x='cluster_dbscan', data=df_clustered)
plt.title("Distribuição dos Clusters pelo DBSCAN")